# Transfer learning experiments

This notebook includes:
1) Implement a transfer learning from ImageNet > 13 diseases > 1 disease. 
2) Compared performance against ImageNet > 1 disease and ImageNet > MURA > 1 disease 

Results for disease corresponding to index 10.

## Imports & global variables

We choose `idx=10` because from previous experiments we have detected a strong signal and a sufficient amount of positive cases. 

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys; sys.path.append("..")
import warnings; warnings.filterwarnings('ignore')

from core import * 
from data_manipulation import Transform, RandomRotation, Flip, RandomCrop, multi_label_2_binary, balance_obs, DataBatches
from utils import save_model, load_model, lr_loss_plot, resize
from architectures import DenseNet121
from train_functions import OptimizerWrapper, TrainingPolicy, FinderPolicy, validate_multilabel, lr_finder, validate_binary, TTA_binary

seed = 42
r_pix = 8
BATCH_SIZE = 16
EPOCHS = 10
TRANSFORMATIONS = [RandomRotation(arc_width=20), Flip(), RandomCrop(r_pix=r_pix)]
PRETRAINED = True
NORMALIZE = True # ImageNet
FREEZE = True
GRADUAL_UNFREEZING = True
DATA = '14diseases'
N_SAMPLES = 1100


BASE_PATH = Path('../..')
PATH = BASE_PATH/'data'
SAVE_RESULTS = './results/'
SAVE_MODELS = './models'
IMG_FOLDER = PATH/'ChestXRay-250'

DISEASE = 'Emphysema'
tgt2idx = {disease: i for i, disease in enumerate([ 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
               'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'])}

# Downstream task - 1 label (small data)

Balance training and testing.

In [4]:
def train(n_epochs, train_dl, valid_dl, model, max_lr=.01, wd=0, alpha=1./ 3,
          save_path=None, unfreeze_during_loop:tuple=None):
    
    if unfreeze_during_loop:
        total_iter = n_epochs*len(train_dl)
        first_unfreeze = int(total_iter*unfreeze_during_loop[0])
        second_unfreeze = int(total_iter*unfreeze_during_loop[1])

    best_loss = np.inf
    cnt = 0
    
    policy = TrainingPolicy(n_epochs=n_epochs, dl=train_dl, max_lr=max_lr)
    optimizer = OptimizerWrapper(model, policy, wd=wd, alpha=alpha)

    for epoch in tqdm_notebook(range(n_epochs), ):
        model.train()
        agg_div = 0
        agg_loss = 0
        train_dl.set_random_choices()
        for x, y in tqdm_notebook(train_dl, leave=False):

            if unfreeze_during_loop:
                if cnt == first_unfreeze: model.unfreeze(1)
                if cnt == second_unfreeze: model.unfreeze(0)

            out = model(x)
            loss = F.binary_cross_entropy_with_logits(input=out.squeeze(), target=y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            batch = y.shape[0]
            agg_loss += batch*loss.item()
            agg_div += batch
            cnt += 1


        val_loss, measure, _ = validate_binary(model, valid_dl)
        print(f'Ep. {epoch+1} - train loss {agg_loss/agg_div:.4f} -  val loss {val_loss:.4f} AUC {measure:.4f}')

        if save_path and val_loss < best_loss:
            save_model(model, save_path)
            best_loss = val_loss

# Comparison on the Downstream task

In [5]:
train_df = pd.read_csv(PATH/'train_df.csv')
valid_df = pd.read_csv(PATH/"val_df.csv")
test_df = pd.read_csv(PATH/"test_df.csv")

train_df = multi_label_2_binary(train_df, tgt2idx[DISEASE])
train_df = train_df.sample(frac=1, random_state=42)

valid_df = multi_label_2_binary(valid_df, tgt2idx[DISEASE])
valid_df = balance_obs(valid_df, amt=None, rate_positive=.5)

test_df = multi_label_2_binary(test_df, tgt2idx[DISEASE])
test_df = balance_obs(test_df, amt=None, rate_positive=.5)

In [6]:
# len(sample_train_df),len(valid_df),len(test_df)

## Without Mixup

In [7]:
p_positive = .10
N = int(p_positive*100)
sample_train_df = balance_obs(train_df, amt=50, rate_positive=p_positive)

In [8]:

train_dl = DataBatches(df=sample_train_df, transforms=TRANSFORMATIONS, shuffle=True,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

valid_dl = DataBatches(df=valid_df, transforms=None, shuffle=False,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

test_dl = DataBatches(df=test_df, transforms=TRANSFORMATIONS, shuffle=False, 
                      img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                      r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

### ImageNet

In [9]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained=True, freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_imagenet.pth'
epochs = 7
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7453 -  val loss 0.7070 AUC 0.4924


Ep. 2 - train loss 0.5137 -  val loss 1.0172 AUC 0.6573


Ep. 3 - train loss 0.2189 -  val loss 1.7676 AUC 0.6992


Ep. 4 - train loss 0.1196 -  val loss 2.3695 AUC 0.7066


Ep. 5 - train loss 0.0625 -  val loss 2.6238 AUC 0.6895


Ep. 6 - train loss 0.0662 -  val loss 2.4579 AUC 0.6941


Ep. 7 - train loss 0.1353 -  val loss 2.3110 AUC 0.7103

TTA loss 2.3872  auc 0.5507  accuracy 0.5000


(2.3872044623225728, 0.5507299633616234, 0.5)

### MURA

In [13]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='MURA', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_MURA.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.5510 -  val loss 0.6647 AUC 0.6755


Ep. 2 - train loss 0.3715 -  val loss 0.8903 AUC 0.7120


Ep. 3 - train loss 0.1658 -  val loss 1.1133 AUC 0.7546


Ep. 4 - train loss 0.0954 -  val loss 1.1130 AUC 0.7619


Ep. 5 - train loss 0.0736 -  val loss 1.0918 AUC 0.7608


Ep. 6 - train loss 0.0524 -  val loss 0.9901 AUC 0.7672

TTA loss 1.0968  auc 0.5818  accuracy 0.5192


(1.0968258416268222, 0.5818462159178135, 0.5192131747483989)

### CheXpert

In [11]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='chexpert', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_chexpert.pth'
epochs = 7
train(epochs, train_dl, valid_dl, model, max_lr=2e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7060 -  val loss 0.6656 AUC 0.6710


Ep. 2 - train loss 0.6303 -  val loss 0.6966 AUC 0.6885


Ep. 3 - train loss 0.4446 -  val loss 0.8156 AUC 0.6779


Ep. 4 - train loss 0.3395 -  val loss 0.9539 AUC 0.6831


Ep. 5 - train loss 0.2900 -  val loss 1.0106 AUC 0.6890


Ep. 6 - train loss 0.2761 -  val loss 1.0829 AUC 0.6887


Ep. 7 - train loss 0.2671 -  val loss 1.0926 AUC 0.6897

TTA loss 1.0753  auc 0.5160  accuracy 0.5000


(1.075325953740923, 0.5160059565612996, 0.5)

### 13 diseases

In [12]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='13diseases', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_13diseases.pth'
epochs = 7
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6034 -  val loss 0.6434 AUC 0.7939


Ep. 2 - train loss 0.5195 -  val loss 0.6688 AUC 0.7889


Ep. 3 - train loss 0.3587 -  val loss 0.7927 AUC 0.7793


Ep. 4 - train loss 0.2648 -  val loss 0.8770 AUC 0.7886


Ep. 5 - train loss 0.2274 -  val loss 0.9058 AUC 0.7954


Ep. 6 - train loss 0.2104 -  val loss 0.9164 AUC 0.7965


Ep. 7 - train loss 0.1971 -  val loss 0.9399 AUC 0.7969

TTA loss 0.8843  auc 0.7480  accuracy 0.5000


(0.8842623812936362, 0.7480414749436863, 0.5)

## With Mixup

In [33]:
train_dl = DataBatches(df=sample_train_df, transforms=TRANSFORMATIONS, shuffle=True,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=True)

valid_dl = DataBatches(df=valid_df, transforms=None, shuffle=False,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

test_dl = DataBatches(df=test_df, transforms=TRANSFORMATIONS, shuffle=False, 
                      img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                      r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

### ImageNet

In [34]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained=True, freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_imagenet_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6965 -  val loss 0.6897 AUC 0.5675


Ep. 2 - train loss 0.6826 -  val loss 0.6838 AUC 0.6732


Ep. 3 - train loss 0.5637 -  val loss 0.6794 AUC 0.7232


Ep. 4 - train loss 0.4600 -  val loss 0.6725 AUC 0.7331


Ep. 5 - train loss 0.5033 -  val loss 0.6569 AUC 0.7396


Ep. 6 - train loss 0.4063 -  val loss 0.6584 AUC 0.7399

TTA loss 0.7895  auc 0.6108  accuracy 0.5517


(0.7894972837500254, 0.6107605664927522, 0.5516925892040256)

### MURA

In [35]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='MURA', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_MURA_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7260 -  val loss 0.6953 AUC 0.5143


Ep. 2 - train loss 0.6434 -  val loss 0.6190 AUC 0.7273


Ep. 3 - train loss 0.4668 -  val loss 0.6522 AUC 0.7424


Ep. 4 - train loss 0.4136 -  val loss 0.6294 AUC 0.7596


Ep. 5 - train loss 0.3641 -  val loss 0.5692 AUC 0.7707


Ep. 6 - train loss 0.3219 -  val loss 0.5708 AUC 0.7680

TTA loss 0.7044  auc 0.6582  accuracy 0.6153


(0.704363933666409, 0.6582000236052598, 0.6152790484903934)

### CheXpert

In [36]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='chexpert', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_chexpert_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=5e-4, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6983 -  val loss 0.7313 AUC 0.3958


Ep. 2 - train loss 0.6936 -  val loss 0.7249 AUC 0.4324


Ep. 3 - train loss 0.6946 -  val loss 0.6948 AUC 0.5658


Ep. 4 - train loss 0.6797 -  val loss 0.6855 AUC 0.6686


Ep. 5 - train loss 0.6796 -  val loss 0.6842 AUC 0.6809


Ep. 6 - train loss 0.6820 -  val loss 0.6837 AUC 0.6904

TTA loss 0.6766  auc 0.7479  accuracy 0.5133


(0.6765524085001017, 0.7479267969085481, 0.5132662397072278)

### 13 diseases

In [38]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='13diseases', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_13diseases_mixup.pth'
epochs = 5
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7006 -  val loss 0.6819 AUC 0.6866


Ep. 2 - train loss 0.6729 -  val loss 0.6392 AUC 0.7957


Ep. 3 - train loss 0.6388 -  val loss 0.6632 AUC 0.8072


Ep. 4 - train loss 0.5955 -  val loss 0.6466 AUC 0.8079


Ep. 5 - train loss 0.5963 -  val loss 0.6414 AUC 0.8089

TTA loss 0.6494  auc 0.7591  accuracy 0.6134


(0.6493558467852338, 0.759102464405863, 0.6134492223238792)

# Results


| Method | test AUC   | test Accuracy (tr = .5) |
|------|------|-------| 
|   ImageNet + Mixup  | 0.611 | 0.552|
|   ImageNet  | 0.625 | 0.624|
|   MURA + Mixup  | 0.659| 0.615|
|   MURA  | 0.674 | 0.606|
|   CheXpert + Mixup  | 0.748| 0.513|
|   CheXpert  | 0.769 | 0.594 |

Mixup seems to don't help in the case of X-rays. 